<a href="https://colab.research.google.com/github/scarecrow021/Beam_Selector_Stress_Deformation/blob/main/Beam_Selector_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
from google.colab import files
import urllib.request
url = 'https://github.com/scarecrow021/Beam_Selector_Stress_Deformation/blob/main/beamLibrary.xlsx'
urllib.request.urlretrieve(url, '/content/beamLibrary.xlsx')

#import os
#if os.path.isfile('/content/beamLibrary.xlsx'):
#    print("File Exists! Carry on")
#else:
#    beamLibFile = files.upload('https://github.com/scarecrow021/Beam_Selector_Stress_Deformation/blob/main/beamLibrary.xlsx')

('/content/beamLibrary.xlsx', <http.client.HTTPMessage at 0x7f7acb6346d0>)

## Back End

### Library Import

In [63]:
import pandas as pd



### Function Definitions


#### Deformation Limit Functions

#### Stress Limit Functions

In [64]:
## Stress Limit Common Functions Definitions

def findStress(M, h, I):                         # Be careful ymax is not h/2 here as in Excel. That is why it is divided by 2 in the formula.
    _stress = (M*(h/(2*1000)))/((I*10**-8)*10**6)
    return _stress


def stressLimitCheck(stressLimit, result, M, beamLib, Iy_columNum, _yes= False, hDim_columNum=1):
    for h in range(0,len(beamLib.index)):
                Stress = findStress(M, beamLib.iat[h,hDim_columNum], beamLib.iat[h,Iy_columNum])
                if Stress < stressLimit:
                    if not _yes:
                        result.append(beamLib.columns[0] + " " + str(beamLib.iat[h,1]) + " ...  Stress: " + str(round(Stress,2)) + " MPa.")
                    else:
                        result.append(beamLib.columns[0] + " " + str(beamLib.iat[h,0]) + " ...  Stress: " + str(round(Stress,2)) + " MPa.")
                    break


def stressLimitCheckLooper(M, result, stressLimit):
    for libNum in range(0,14):
        beamLib = pd.read_excel('/content/beamLibrary.xlsx', libNum)
        if libNum in [8,13,14]:
            stressLimitCheck(stressLimit, result, M, beamLib, 5, True)
        elif libNum == 12:
            stressLimitCheck(stressLimit, result, M, beamLib, 4, True)
        elif libNum in [7,9,10,11]:
            stressLimitCheck(stressLimit, result, M, beamLib, 3, True)
        elif libNum in range(0,7):
            stressLimitCheck(stressLimit, result, M, beamLib, 6)
    return result

#----------------------------------------------------------------------------------------------------------------------------------------------#

## All Case Stress Limit Functions Defitions

# The stress formula is same in all cases: M*y/I
# Since the bending moment is different for all cases, it is best to calculate 
# the bending moment in the function here and input it to the called function.

def case1_stress(stressLimit):
    M = force*lengthOfBeam/4 # Bending Moment
    result = ["You selected Case 1, Stress Limit", "The minimum required beam and the stress are: "] # Result String Array
    _result = stressLimitCheckLooper(M, result, stressLimit)
    return _result

def case2_stress(stressLimit):
    M = force*lengthOfBeam # Bending Moment
    result = ["You selected Case 2, Stress Limit", "The minimum required beam and the stress are: "] # Result String Array
    _result = stressLimitCheckLooper(M, result, stressLimit)
    return _result

def case3_stress(stressLimit):
    M = force*lengthOfBeam/8 # Bending Moment
    result = ["You selected Case 2, Stress Limit", "The minimum required beam and the stress are: "] # Result String Array
    _result = stressLimitCheckLooper(M, result, stressLimit)
    return _result

def case4_stress(stressLimit):
    M = torque # Bending Moment
    result = ["You selected Case 2, Stress Limit", "The minimum required beam and the stress are: "] # Result String Array
    _result = stressLimitCheckLooper(M, result, stressLimit)
    return _result

def case5_stress(stressLimit):
    M = torque # Bending Moment
    result = ["You selected Case 2, Stress Limit", "The minimum required beam and the stress are: "] # Result String Array
    _result = stressLimitCheckLooper(M, result, stressLimit)
    return _result

def case6_stress(stressLimit, length_a):
    M = force*length_a # Bending Moment
    result = ["You selected Case 2, Stress Limit", "The minimum required beam and the stress are: "] # Result String Array
    _result = stressLimitCheckLooper(M, result, stressLimit)
    return _result

def case7_stress(stressLimit, lengthOfBeam, length_b):
    M = force*(lengthOfBeam - length_b)*length_b/lengthOfBeam # Bending Moment
    result = ["You selected Case 2, Stress Limit", "The minimum required beam and the stress are: "] # Result String Array
    _result = stressLimitCheckLooper(M, result, stressLimit)
    return _result

# **Input Form** 

In [65]:
caseNumber = 1 #@param ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14'] {type:"raw"}
checkFor = "Stress" #@param ["Stress", "Deformation"]
force =  1000#@param {type:"number"}
torque = 0 #@param {type:"number"}
lengthOfBeam =  4#@param {type:"number"}
length_a = 0 #@param {type:"number"}
length_b = 0 #@param {type:"number"}
length_c = 0 #@param {type:"number"}
youngsModulus = 203 #@param {type:"number"}
stressLimit =  200#@param {type:"number"}
deformationLimit = 0 #@param {type:"number"}


# Interface

In [66]:
def caseOne():
    if checkFor == "Stress": _result = case1_stress(stressLimit)
    return _result

def caseTwo():
    if checkFor == "Stress": _result = case2_stress(stressLimit)
    return _result

def caseCaller():
    
    if caseNumber == 1:
        _result = caseOne()
    elif caseNumber == 2:
        _result = caseTwo()
    else:
        _result = "Invalid"
    return _result


# **RESULT**

In [67]:
result = caseCaller()
print(*result,sep='\n')

You selected Case 1, Stress Limit
The minimum required beam and the stress are: 
HEB 100 ...  Stress: 11.12 MPa.
HEA 96 ...  Stress: 13.75 MPa.
HEM 120 ...  Stress: 5.25 MPa.
IPE 80 ...  Stress: 49.94 MPa.
IPN 80 ...  Stress: 51.41 MPa.
UPE 80 ...  Stress: 37.38 MPa.
UPN 40 ...  Stress: 141.84 MPa.
Equal L L 140 x 140 x 18 ...  Stress: 188.17 MPa.
Unequal L L 120 x 80 x 8 ...  Stress: 26.58 MPa.
Square Tube CF 1997 40 x 4 ...  Stress: 180.18 MPa.
Square Tube HF 1997 40 x 3.2 ...  Stress: 196.08 MPa.
Square Tube HF 2006 50 x 10 ...  Stress: 198.41 MPa.
Circular Tube 48.3 x 4 ...  Stress: 175.0 MPa.
Rectangular Tube CF 50 x 30 x 3 ...  Stress: 194.86 MPa.
